<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip -q install transformers
!pip -q install sentencepiece

     |████████████████████████████████| 1.8MB 956kB/s 
     |████████████████████████████████| 2.9MB 42.5MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
     |████████████████████████████████| 1.2MB 3.0MB/s 


In [2]:
import pandas as pd
import numpy as np
from transformers import (
    pipeline, 
    AutoConfig, 
    AutoTokenizer, 
    AutoModel, 
    AutoModelForTokenClassification
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model = AutoModelForTokenClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model.eval()
print()

In [5]:
albert_ner = pipeline('ner', model=model, tokenizer=tokenizer)

In [6]:
! rm -rf data
! wget -q --show-progress https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data{1..10}.txt
! mkdir data && mv persian-ner* data/

persian-ner-twitter 100%[===================>] 276.22K  --.-KB/s    in 0.08s   
persian-ner-twitter 100%[===================>] 245.48K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 308.37K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 226.92K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 296.95K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 217.98K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 287.11K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 263.38K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 298.06K  --.-KB/s    in 0.001s  
persian-ner-twitter 100%[===================>] 103.19K  --.-KB/s    in 0s      


# Convert to Text

In [10]:
from pathlib import Path
import re

def convert_lines_to_text(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [11]:
texts, tags = convert_lines_to_text('data/persian-ner-twitter-data1.txt')

In [12]:
print(texts[0])
print(tags[0])

['خرداد', '1399', '«', 'کیهان', '»', 'میراث', 'سناتور', 'مصطفی', 'مصباح\u200cزاده', '78', 'ساله', 'شد', 'کیهان', 'لندن', 'تنها', 'رسانه', 'ایرانی', 'در', 'تبعید', 'است', 'که', '«', 'از', 'جنگ', 'دوم', 'جهانی', 'تاکنون', 'قدمت', 'و', 'یک', 'موسسه', 'غصب', 'شده', 'در', 'تهران', '»', 'دارد']
['O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'B-LOC', 'O', 'O', 'B-NAT', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVE', 'I-EVE', 'I-EVE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O']


In [13]:
s = ''
for word in texts[0]:
  s += word + ' '
s

'خرداد 1399 « کیهان » میراث سناتور مصطفی مصباح\u200cزاده 78 ساله شد کیهان لندن تنها رسانه ایرانی در تبعید است که « از جنگ دوم جهانی تاکنون قدمت و یک موسسه غصب شده در تهران » دارد '

# Benchmark

In [ ]:
for idx, item in enumerate(tokenizer.tokenize(s)):
  print(item)
  if idx < len(texts[0]):
    print(texts[0][idx])
  print('----------')

▁خرداد
خرداد
----------
▁
1399
----------
1399
«
----------
▁
کیهان
----------
«
»
----------
▁کیهان
میراث
----------
▁
سناتور
----------
»
مصطفی
----------
▁میراث
مصباح‌زاده
----------
▁سناتور
78
----------
▁مصطفی
ساله
----------
▁مصباح
شد
----------
▁زاده
کیهان
----------
▁
لندن
----------
78
تنها
----------
▁ساله
رسانه
----------
▁شد
ایرانی
----------
▁کیهان
در
----------
▁لندن
تبعید
----------
▁تنها
است
----------
▁رسانه
که
----------
▁ایرانی
«
----------
▁در
از
----------
▁تبعید
جنگ
----------
▁است
دوم
----------
▁که
جهانی
----------
▁
تاکنون
----------
«
قدمت
----------
▁از
و
----------
▁جنگ
یک
----------
▁دوم
موسسه
----------
▁جهانی
غصب
----------
▁تاکنون
شده
----------
▁قدمت
در
----------
▁و
تهران
----------
▁یک
»
----------
▁موسسه
دارد
----------
▁غصب
----------
▁شده
----------
▁در
----------
▁تهران
----------
▁
----------
»
----------
▁دارد
----------


In [ ]:
texts[0][]

'1399'

In [ ]:
s[13:18]

'کیهان'

In [ ]:
pred_list = albert_ner(s)
for pred in pred_list:
  print("word:", pred['word'])
  print("entity:", pred['entity'])
  print("index:", pred['index'])
  
  print("s[start:end]:",s[pred['start']:pred['end']])
  print("text[index]:",texts[0][pred['index']/2])
  print("tags[index]:",tags[0][pred['index']])
  print("-----------")

word: ▁کیهان
entity: B-organization
index: 6
s[start:end]: کیهان
text[index]: سناتور
tags[index]: O
-----------
word: ▁مصطفی
entity: B-person
index: 11
s[start:end]: مصطفی
text[index]: شد
tags[index]: O
-----------
word: ▁مصباح
entity: I-person
index: 12
s[start:end]: مصباح
text[index]: کیهان
tags[index]: B-ORG
-----------
word: ▁زاده
entity: I-person
index: 13
s[start:end]: زاده
text[index]: لندن
tags[index]: B-LOC
-----------
word: ▁کیهان
entity: B-organization
index: 18
s[start:end]: کیهان
text[index]: تبعید
tags[index]: O
-----------
word: ▁لندن
entity: I-organization
index: 19
s[start:end]: لندن
text[index]: است
tags[index]: O
-----------
word: ▁تهران
entity: B-location
index: 41
s[start:end]: تهران


IndexError: ignored

# Benchmark2

In [ ]:
!cat data/dev.txt.tmp | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp
# !cat NER-de-dev.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp
# !cat NER-de-test.tsv | grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp

In [ ]:
!git clone https://github.com/huggingface/transformers
!pip install transformers/

In [ ]:
!pip install sentencepiece

In [ ]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

In [ ]:
# !python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > train.txt
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 128 > dev.txt
# !python3 preprocess.py test.txt.tmp m3hrdadfi/albert-fa-base-v2-ner 128 > test.txt

In [ ]:
!pip install -r transformers/examples/token-classification/requirements.txt

In [ ]:
!mv dev.txt data/

In [ ]:
!cat data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [ ]:
!python3 transformers/examples/legacy/token-classification/run_ner.py --data_dir data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  128 \
--save_steps 750 \
--seed 1 \
--do_eval

In [ ]:
!pip install conllu

In [ ]:
import pandas as pd

In [ ]:
with open('data-fa/train.csv', 'r+') as file:
  s = ''
  for line in file.readlines():
    s += line.replace(',', '\t')
  file.seek = 0
  file.write(s)

In [ ]:
!cat data-fa/train.csv

In [ ]:
!curl -L 'https://drive.google.com/file/d/1Jjhbal535VVz2ap4v4r_rN1UEHTdLK5P/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > train.txt.tmp
!curl -L 'https://drive.google.com/file/d/1ZfRcQThdtAR5PPRjIDtrVP7BtXSCUBbm/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp
!curl -L 'https://drive.google.com/file/d/1u9mb7kNJHWQCWyweMDRMuTFoOHOfeBTH/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp